# Find LSOAs in catchment area of each hospital

In [1]:
import pandas as pd

## Import data

Travel time matrix between each LSOA and each hospital:

In [2]:
df_time_lsoa_hospital = pd.read_csv('../data_tabular/lsoa_travel_time_matrix_calibrated.csv', index_col='LSOA')

In [3]:
df_time_lsoa_hospital.columns

Index(['B152TH', 'B714HJ', 'B95SS', 'BA13NG', 'BA214AT', 'BB23HH', 'BD96RJ',
       'BH77DW', 'BL97TD', 'BN112DH',
       ...
       'TN240LZ', 'TQ27AA', 'TR13LQ', 'TS198PE', 'TS43BW', 'WD180HB', 'WF14DG',
       'WR51DD', 'WV100QP', 'YO318HE'],
      dtype='object', length=125)

In [4]:
df_time_lsoa_hospital.index

Index(['Adur 001A', 'Adur 001B', 'Adur 001C', 'Adur 001D', 'Adur 001E',
       'Adur 001F', 'Adur 002A', 'Adur 002B', 'Adur 002C', 'Adur 002D',
       ...
       'York 022F', 'York 023A', 'York 023B', 'York 023C', 'York 024A',
       'York 024B', 'York 024C', 'York 024D', 'York 024E', 'York 024F'],
      dtype='object', name='LSOA', length=34752)

LSOA names and codes table:

Created in `link_LSOAs_to_ONS_regions.ipynb`

In [5]:
df_regions = pd.read_csv('../data_tabular/LSOA_regions.csv')

In [6]:
df_regions.columns

Index(['LSOA11CD', 'LSOA11NM', 'LSOA11LONG', 'LSOA11LAT', 'CCG19CD', 'CCG19NM',
       'STP19CD', 'STP19NM', 'LHB20CD', 'LHB20NM', 'LHB20NMW', 'LAD17CD',
       'LAD17NM', 'SCN17CD', 'SCN17NM', 'RGN11CD', 'RGN11NM'],
      dtype='object')

In [7]:
df_regions['LSOA11NM']

0        Adur 001A
1        Adur 001B
2        Adur 001C
3        Adur 001D
4        Adur 001E
           ...    
34747    York 024B
34748    York 024C
34749    York 024D
34750    York 024E
34751    York 024F
Name: LSOA11NM, Length: 34752, dtype: object

Check whether this list is identical to the list in the other file:

In [8]:
(df_regions['LSOA11NM'] == df_time_lsoa_hospital.index).all()

True

List of stroke teams and which treatments they can provide:

In [9]:
df_stroke_teams = pd.read_csv('../data_tabular/stroke_hospitals_2022.csv')

In [10]:
df_stroke_teams.columns

Index(['Postcode', 'Hospital_name', 'Use_IVT', 'Use_MT', 'Use_MSU', 'Country',
       'Strategic Clinical Network', 'Health Board / Trust', 'Stroke Team',
       'SSNAP name', 'Admissions 21/22', 'Thrombolysis', 'ivt_rate', 'Easting',
       'Northing', 'long', 'lat', 'Neuroscience',
       '30 England Thrombectomy Example', 'hospital_city', 'Notes'],
      dtype='object')

In [11]:
MT_units = df_stroke_teams['Postcode'][df_stroke_teams['Use_MT'] == 1].values

MT_units

array(['RM70AG', 'E11BB', 'SW66SX', 'SE59RW', 'SW170QT', 'NW12BU',
       'NG72UH', 'CB20QQ', 'B152TH', 'CV22DX', 'ST46QG', 'L97AL',
       'PR29HT', 'M68HD', 'NE14LP', 'HU32JZ', 'LS13EX', 'S102JF',
       'BN25BE', 'BS105NB', 'PL68DH', 'OX39DU', 'SO166YD', 'CF144XW'],
      dtype=object)

## Assign LSOAs to hospitals

Put the final data into this new dataframe:

In [12]:
df_lsoa_hospitals = pd.DataFrame()

Find information about the nearest stroke team to each LSOA:

In [13]:
# Nearest stroke team:
# The smallest time in each row:
df_lsoa_hospitals['time_nearest_stroke_team'] = df_time_lsoa_hospital.min(axis='columns')
# The name of the column containing the smallest time in each row:
df_lsoa_hospitals['postcode_nearest_stroke_team'] = df_time_lsoa_hospital.idxmin(axis='columns')

# Merge in unit names for nearest stroke teams:
df_lsoa_hospitals['lsoa'] = df_lsoa_hospitals.index
df_lsoa_hospitals = pd.merge(
    df_lsoa_hospitals,
    df_stroke_teams[['Postcode', 'Stroke Team', 'SSNAP name']],
    left_on='postcode_nearest_stroke_team',
    right_on='Postcode'
)
df_lsoa_hospitals = df_lsoa_hospitals.set_index('lsoa')
df_lsoa_hospitals = df_lsoa_hospitals.drop('Postcode', axis=1)
df_lsoa_hospitals = df_lsoa_hospitals.rename(columns={
    'Stroke Team': 'name_nearest_stroke_team',
    'SSNAP name': 'ssnap_name_nearest_stroke_team',
})

Find information about the nearest MT uni to each LSOA:

In [14]:
# Nearest MT unit:
# The smallest time in each row:
df_lsoa_hospitals['time_nearest_MT'] = df_time_lsoa_hospital[MT_units].min(axis='columns')
# The name of the column containing the smallest time in each row:
df_lsoa_hospitals['postcode_nearest_MT'] = df_time_lsoa_hospital[MT_units].idxmin(axis='columns')
# # Merge in unit names for nearest MT unit:
df_lsoa_hospitals['lsoa'] = df_lsoa_hospitals.index
df_lsoa_hospitals = pd.merge(
    df_lsoa_hospitals,
    df_stroke_teams[['Postcode', 'Stroke Team', 'SSNAP name']],
    left_on='postcode_nearest_MT',
    right_on='Postcode'
)
df_lsoa_hospitals = df_lsoa_hospitals.drop('Postcode', axis=1)
df_lsoa_hospitals = df_lsoa_hospitals.rename(columns={
    'Stroke Team': 'name_nearest_MT',
    'SSNAP name': 'ssnap_name_nearest_MT',
})

Add in extra identifiers - LSOA code name `LSOA11CD` from the ONS data.

In [15]:
# Add in LSOA code:
df_lsoa_hospitals = pd.merge(
    df_lsoa_hospitals,
    df_regions[['LSOA11NM', 'LSOA11CD']],
    left_on='lsoa',
    right_on='LSOA11NM'
)
df_lsoa_hospitals = df_lsoa_hospitals.drop('lsoa', axis=1)

Reorder columns:

In [16]:
df_lsoa_hospitals = df_lsoa_hospitals[[
    'LSOA11NM',
    'LSOA11CD',
    'time_nearest_stroke_team',
    'postcode_nearest_stroke_team',
    'name_nearest_stroke_team',
    'ssnap_name_nearest_stroke_team',
    'time_nearest_MT',
    'postcode_nearest_MT',
    'name_nearest_MT',
    'ssnap_name_nearest_MT',
]]

Final combined dataframe:

In [17]:
df_lsoa_hospitals

,LSOA11NM,LSOA11CD,time_nearest_stroke_team,postcode_nearest_stroke_team,name_nearest_stroke_team,ssnap_name_nearest_stroke_team,time_nearest_MT,postcode_nearest_MT,name_nearest_MT,ssnap_name_nearest_MT
0,Adur 001A,E01031349,17.6,BN25BE,"Royal Sussex County Hospital, Brighton",Royal Sussex County Hospital,17.6,BN25BE,"Royal Sussex County Hospital, Brighton",Royal Sussex County Hospital
1,Adur 004A,E01031347,17.6,BN25BE,"Royal Sussex County Hospital, Brighton",Royal Sussex County Hospital,17.6,BN25BE,"Royal Sussex County Hospital, Brighton",Royal Sussex County Hospital
2,Adur 004B,E01031348,16.5,BN25BE,"Royal Sussex County Hospital, Brighton",Royal Sussex County Hospital,16.5,BN25BE,"Royal Sussex County Hospital, Brighton",Royal Sussex County Hospital
3,Brighton and Hove 001A,E01016924,18.7,BN25BE,"Royal Sussex County Hospital, Brighton",Royal Sussex County Hospital,18.7,BN25BE,"Royal Sussex County Hospital, Brighton",Royal Sussex County Hospital
4,Brighton and Hove 001B,E01016926,17.6,BN25BE,"Royal Sussex County Hospital, Brighton",Royal Sussex County Hospital,17.6,BN25BE,"Royal Sussex County Hospital, Brighton",Royal Sussex County Hospital
...,...,...,...,...,...,...,...,...,...,...
34747,Torbay 019C,E01015212,9.0,TQ27AA,Torbay Hospital,Torbay Hospital,54.1,PL68DH,"Derriford Hospital, Plymouth",Derriford Hospital
34748,Torbay 019D,E01015213,10.1,TQ27AA,Torbay Hospital,Torbay Hospital,52.0,PL68DH,"Derriford Hospital, Plymouth",Derriford Hospital
34749,Torbay 019E,E01015207,10.1,TQ27AA,Torbay Hospital,Torbay Hospital,50.9,PL68DH,"Derriford Hospital, Plymouth",Derriford Hospital
34750,Torbay 019F,E01015208,11.2,TQ27AA,Torbay Hospital,Torbay Hospital,49.8,PL68DH,"Derriford Hospital, Plymouth",Derriford Hospital


Save to file:

In [18]:
df_lsoa_hospitals.to_csv('../data_tabular/lsoa_nearest_stroke_team.csv', index=False)